In [1]:
import re
import nltk
import os
import pandas as pd
import numpy as np 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/data_skripsi'

/content/drive/My Drive/Colab Notebooks/data_skripsi


In [19]:
!ls

 actual_word			   kata_dasar_db.txt
 hasil_idris			   PENGUJIAN_IDRISnew.ipynb
 hasil_inidris			   PENGUJIAN_INIDRISnew.ipynb
'Hasil perbandingan Idris.csv'	  'Perbandingan Stemming Idris.csv'
'Hasil perbandingan Inidris.csv'  'Perbandingan Stemming Inidris.csv'
 IDRIS.ipynb			   stopwords2.txt
 INIDRIS.ipynb			   stopwords.txt
 katadasarbahasaindonesia.txt	   test_data


In [20]:
stopword = open("stopwords2.txt", 'r').read() 
stopword = stopword.split("\n")
stopword[:5]

['ada', 'adalah', 'adanya', 'adapun', 'agak']

In [21]:
kata_dasar = open("kata_dasar_db.txt", 'r').read()
kata_dasar = kata_dasar.split("\n")

kamus = []
for i in kata_dasar:
    kata = "{}".format(i)
    kata = kata.replace("(Nomina)", "")
    kata = kata.replace("(Adjektiva)", "")
    kata = kata.replace("(Adverbia)", "")
    kata = kata.replace("(Numeralia)", "")
    kata = kata.replace("(Pronomina)", "")
    kata = kata.replace("(Verba)", "")
    kata = kata.replace("(Preposisi)", "")
    kata = kata.replace("(Interjeksi)", "")
    kata = kata.replace("(Konjungsi)", "")
    kata = kata.replace("(Lain-lain)", "")
    kata = kata.replace("\n", "")
    kata = kata.rstrip()
    kamus.append(kata)
kamus[:5]

['a', 'ab', 'aba', 'aba-aba', 'abad']

In [22]:
# cek kata dasar di kamus
def cekKamus_idris(kata):
    if kata in kamus:
        return kata
    else:
        status = 0
    return status

In [23]:
# fungsi untuk menghapus suffix seperti -ku, -mu, -kah, dsb
def del_inflection_suffixes_idris(kata):
    kataAsal = kata
    if(re.search(r"([kl]ah|pun)$", kata, re.IGNORECASE)):
        _kata = re.sub(r"([kl]ah|pun)$", '', kata, re.IGNORECASE)
        if(cekKamus_idris(_kata)):
            return _kata
        else:
            kata = _kata
    if(re.search(r'([km]u|nya)$', kata, re.IGNORECASE)):
        _kata = re.sub(r'([km]u|nya)$', '', kata, re.IGNORECASE)
        if(cekKamus_idris(_kata)):
            return _kata
    return kataAsal

In [24]:
# Cek Prefix Disallowed Sufixes (Kombinasi Awalan dan Akhiran yang tidak diizinkan)
def cek_prefix_disallowed_sufiks_idris(kata):
    kataAsal = kata
    # r"^(ber)[[:alpha:]](lah|an)$"
    if(re.search(r"^(ber)[^\W\d_](lah|an)$", kata, re.IGNORECASE)):
        _kata = re.sub(r"^(ber)", '', kata, re.IGNORECASE)
        return _kata
    # ^(men|di|pe|ter)[[:alpha:]](i)
    if (re.search(r"^(men|di|pe|ter)[^\W\d_](i)$", kata, re.IGNORECASE)):
        _kata = re.sub(r"^(men|di|pe|ter)", '', kata, re.IGNORECASE)
        return _kata
    return kataAsal

In [25]:
# Hapus Derivation Suffixes ("-i", "-an" atau "-kan")
def del_derivation_suffixes_idris(kata):
    if(re.search(r"(i|an)$", kata, re.IGNORECASE)):
        _kata = re.sub(r"(i|an)$", '', kata, re.IGNORECASE)
        if(cekKamus_idris(_kata)):
            return _kata
        elif(re.search(r"(kan)$", kata)):
            _kata = re.sub(r"(kan)$", '', kata, re.IGNORECASE)
            if(cekKamus_idris(_kata)):
                return _kata
            else:
                _kata = del_derivation_prefix_idris(_kata)
                return _kata

In [26]:
# Hapus Derivation Prefix ("di-", "ke-", "se-", "te-", "be-", "me-", atau "pe-")
def del_derivation_prefix_idris(kata):
    kataAsal = kata
    if(re.search(r"^(diper)", kata, re.IGNORECASE)):
        _kata = re.sub(r"^(diper)", '', kata, re.IGNORECASE)
        return _kata

    if(re.search(r"^(di|[ks]e)", kata, re.IGNORECASE)):
        _kata = re.sub(r"^(di|[ks]e)", '', kata, re.IGNORECASE)
        return _kata

    if(re.search(r"^(ke[bt]er)", kata, re.IGNORECASE)):
        _kata = re.sub(r"^(ke[bt]er)", '', kata, re.IGNORECASE)
        return _kata

    if(re.search(r"^([bt]e)", kata, re.IGNORECASE)):
        _kata = re.sub(r"^([bt]e[lr])", '', kata, re.IGNORECASE)
        if(cekKamus_idris(_kata)):
            return _kata
        _kata = re.sub(r"^([bt]e)", '', kata, re.IGNORECASE)
        return _kata

    if(re.search(r"^([mp]e)", kata, re.IGNORECASE)):
        _kata = re.sub(r"^([mp]e)", '', kata, re.IGNORECASE)
        if(cekKamus_idris(_kata)):
            return _kata
        else:
            if(re.search(r"^(memper)", kata, re.IGNORECASE)):
                _kata = re.sub(r"^(memper)", '', kata, re.IGNORECASE)
                if(cekKamus_idris(_kata)):
                    return _kata

            if(re.search(r"^([mp]eng)", kata, re.IGNORECASE)):
                _kata = re.sub(r"^([mp]eng)", '', kata, re.IGNORECASE)
                if(cekKamus_idris(_kata)):
                    return _kata
                else:
                    _kata = re.sub(r"([mp]eng)", 'k', kata, re.IGNORECASE)

            if(re.search(r"^([mp]eny)", kata, re.IGNORECASE)):
                _kata = re.sub(r"^([mp]eny)", 's', kata, re.IGNORECASE)

            if(re.search(r"^([mp]e[lr])", kata, re.IGNORECASE)):
                _kata = re.sub(r"^([mp]e[lr])", '', kata, re.IGNORECASE)

            if(re.search(r"^([mp]en)", kata, re.IGNORECASE)):
                _kata = re.sub(r"^([mp]en)", 't', kata, re.IGNORECASE)
                if(cekKamus_idris(_kata)):
                    return _kata
                else:
                    _kata = re.sub(r"^([mp]en)", '', kata, re.IGNORECASE)

            if(re.search(r"^([mp]em)", kata, re.IGNORECASE)):
                _kata = re.sub(r"^([mp]em)", '', kata, re.IGNORECASE)
                if(cekKamus_idris(_kata)):
                    return _kata
                else:
                    _kata = re.sub(r"^([mp]em)", 'p', kata, re.IGNORECASE)

            return _kata
    return kataAsal

In [27]:
def step2_idris(kata):
    kataAsal = kata
    kata = cek_prefix_disallowed_sufiks_idris(kata)
    if(cekKamus_idris(kata)):
        return kata
    else:
        kata = del_derivation_prefix_idris(kata)
        if(cekKamus_idris(kata)):
            return kata
        else:
            kata = del_inflection_suffixes_idris(kata)
            if(cekKamus_idris(kata)):
                return kata
            else:
                kata = del_derivation_suffixes_idris(kata)
                # print("4"+str(kata))
                if(cekKamus_idris(kata)):
                    return kata
                else:
                    return kataAsal+" "+"(Kata Salah)"

In [28]:
def stemming_idris(kata):
    kataAsal = kata
    # cekKamus = cekKamus_idris(kata)
    if (cekKamus_idris(kata) != 0):
        return kata
    else:
        kata = step2_idris(kata)
        return kata

## CASE FOLDING

In [29]:
def Case_folding(casefold):
    casefold = casefold.lower()
    # Remove angka termasuk angka yang berada dalam string
    # Remove non ASCII chars
    casefold = re.sub(r'[^\x00-\x7f]', r'', casefold)
    casefold = re.sub(r'(\\u[0-9A-Fa-f]+)', r'', casefold)
    casefold = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", casefold)
    casefold = re.sub(r'\\u\w\w\w\w', '', casefold)
    # Remove link web
    casefold = re.sub(r'http\S+', '', casefold)
    # Remove @username
    casefold = re.sub('@[^\s]+', '', casefold)
    # Remove #tagger
    casefold = re.sub(r'#([^\s]+)', '', casefold)
    # Remove simbol, angka dan karakter aneh
    casefold = re.sub(r"[.,:;+!\-_<^/=?\"'\(\)\d\*]", " ", casefold)
    return casefold

## TOKENIZING

In [30]:
# Tokenizing = Kata.split()
# print(Tokenizing)

## STOPWORD REMOVING

In [31]:
def removeStopword(remove_stop):
    Stopword_data = open('stopwords2.txt', 'r').read().split()
    Kata_baru = []
    content = []
    filteredtext = [word for word in remove_stop if word not in Stopword_data]
    content.append(" ".join(filteredtext))
    remove_stop = content
    return remove_stop

## FUNGSI PERBANDINGAN HASIL

In [32]:
def create_output(string_split, hasil_stem, actual_word, filename):
    #membuat dataframe dengan nama kolom dibawah ini
    df = pd.DataFrame(columns =  ["Kata Asli", "Stemming Idris", "Stemming Asli", "Hasil"])
    for i in range(len(string_split)):
        #jika kata tersebut adalah kata asing, nama kota, nama orang dsb
        if '(' in actual_word[i] and string_split[i] == hasil_stem[i] and hasil_stem[i] == actual_word[i][:actual_word[i].find('(')]:
            new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i][:actual_word[i].find('(')], 'Hasil':'Bukan Kata Berimbuhan'}
            df = df.append(new_row, ignore_index=True)
        
        elif'(' in actual_word[i] and string_split[i] == hasil_stem[i] and hasil_stem[i] != actual_word[i][:actual_word[i].find('(')]:
            new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i][:actual_word[i].find('(')], 'Hasil':'Unchange'}
            df = df.append(new_row, ignore_index=True)
            
        #sudah dari awal kata tersebut adalah kata dasar
        elif string_split[i] == hasil_stem[i] and hasil_stem[i] == actual_word[i]:
            new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Kata Dasar'}
            df = df.append(new_row, ignore_index=True)
    
        #kata tersebut berhasil distemming dengan baik
        elif string_split[i] != hasil_stem[i] and hasil_stem[i] == actual_word[i]:
            new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Benar'}
            df = df.append(new_row, ignore_index=True)

        #Kata tersebut tidak berubah dari awal (tidak terstemming)
        elif string_split[i] == hasil_stem[i] and hasil_stem[i] != actual_word[i]:
            new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Unchange'}
            df = df.append(new_row, ignore_index=True)
        
        #kata tersebut berhasil distemming tetapi hasilnya salah
        elif string_split[i] != hasil_stem[i] and len(hasil_stem[i]) == len(actual_word[i]):
            new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Spelling exception'}
            df = df.append(new_row, ignore_index=True)
        
        #kata tersebut berhasil distemming tetapi hasilnya overstemming
        elif string_split[i] != hasil_stem[i] and len(hasil_stem[i]) < len(actual_word[i]):
            new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Overstemming'}
            df = df.append(new_row, ignore_index=True)
        
        #kata tersebut berhasil distemming tetapi hasilnya understemming
        elif string_split[i] != hasil_stem[i] and len(hasil_stem[i]) > len(actual_word[i]):
            new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Understemming'}
            df = df.append(new_row, ignore_index=True)

        #jika tidak masuk semua kondisi diatas
        else:
            new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'None'}
            df = df.append(new_row, ignore_index=True)
    
    #export ke file csv
    hasil_path = os.path.join(os.getcwd(), 'hasil_idris')
    df.to_csv(os.path.join(hasil_path, 'Perbandingan Stemming Idris '+str(file[:-4])+'.csv'), index=False)
    return df

## HASIL

In [33]:
#dapatkan list file dari test data
test_data_path = os.path.join(os.getcwd(), 'test_data')
test_data_filelist = os.listdir(test_data_path)
test_data_filelist_sorted = sorted(test_data_filelist, key=lambda x: int(x.split('.')[0]))

#path dari actual word
actual_data_path = os.path.join(os.getcwd(), 'actual_word')

#path hasil idris
hasil_path = os.path.join(os.getcwd(), 'hasil_idris')

for file in test_data_filelist_sorted:
    print(file)
    
    #preprocessing
    txt = open(os.path.join(test_data_path, file), encoding='cp1252')
    # txt = open(os.path.join(test_data_path, file))
    txt = Case_folding(txt.read())
    txt = txt.split()
    len_txt = len(txt)
    txt = removeStopword(txt)
    
    hasil_stem_idris = []

    Kata_baru = txt[0]

    #stemming idris
    string_split = Kata_baru.split(' ')
    with open(os.path.join(hasil_path, 'Hasil Stemming Idris '+str(file[:-4])+'.txt'), 'w') as i:
        for a in string_split:
            st = stemming_idris(a)
            hasil_stem_idris.append(st)
            i.write(st)
            i.write('\n')
    
    # hasil_stem_idris
#     jumlah_kata = len(string_split)
    jumlah_stem = len(hasil_stem_idris)
    kata_benar = sum('(Kata Salah)' not in x for x in hasil_stem_idris)
    kata_salah = sum('(Kata Salah)' in x for x in hasil_stem_idris)
    acc = round((kata_benar/jumlah_stem)*100, 3)

    print("  jumlah total Kata \t=", str(len_txt))
    print("  jumlah stemming Kata \t=", str(jumlah_stem))
    print("  jumlah Kata benar \t=", str(kata_benar))
    print("  jumlah Kata salah \t=", str(kata_salah))
    print("  Akurasi stemming \t= {}%\n".format(acc))
    
    #menghilangkan kata yang mengandung tulisan (Kata Salah) dari list ex: yunani (Kata Salah) ==> yunani
    hasil_stem_idris_dasar = [x[:-13] if 'Salah' in x else x for x in hasil_stem_idris]
    
    #mengambil list actual word (kata yang benar)
    actual_word = open(os.path.join(actual_data_path, file))
    actual_word = [line.rstrip('\n') for line in actual_word]
    
    #perbandingan hasil stemming inidris dengan actual word
    df = create_output(string_split, hasil_stem_idris_dasar, actual_word, file)
    
    print("  Hasil Perbandingan Dengan Data Sebenarnya")
    print("   Kata dasar benar tanpa imbuhan \t\t\t\t(benar) =", len(df[df.Hasil == 'Kata Dasar']))
    print("   Kata dasar hasil stemming  \t\t\t\t\t(benar) =", len(df[df.Hasil == 'Benar']))
    print("   Bukan Kata Berimbuhan \t\t\t\t\t(salah) =", len(df[df.Hasil == 'Bukan Kata Berimbuhan']))
    print("   Kata yang tidak berubah(Unchange) \t\t\t\t(salah) =", len(df[df.Hasil == 'Unchange']))
    print("   Kata hasil Stemming yang salah(spelling exception) \t\t(benar) =", len(df[df.Hasil == 'Spelling exception']))
    print("   Kata hasil stemming yg understemming \t\t\t(benar) =", len(df[df.Hasil == 'Understemming']))
    print("   Kata hasil stemming yg overstemming \t\t\t\t(benar) =", len(df[df.Hasil == 'Overstemming']), '\n')
        

1.Percy Jackson dan Pencuri Petir.txt
  jumlah total Kata 	= 239
  jumlah stemming Kata 	= 123
  jumlah Kata benar 	= 96
  jumlah Kata salah 	= 27
  Akurasi stemming 	= 78.049%

  Hasil Perbandingan Dengan Data Sebenarnya
   Kata dasar benar tanpa imbuhan 				(benar) = 66
   Kata dasar hasil stemming  					(benar) = 28
   Bukan Kata Berimbuhan 					(salah) = 8
   Kata yang tidak berubah(Unchange) 				(salah) = 19
   Kata hasil Stemming yang salah(spelling exception) 		(benar) = 1
   Kata hasil stemming yg understemming 			(benar) = 0
   Kata hasil stemming yg overstemming 				(benar) = 1 

2.The Hunger Games.txt
  jumlah total Kata 	= 261
  jumlah stemming Kata 	= 146
  jumlah Kata benar 	= 118
  jumlah Kata salah 	= 28
  Akurasi stemming 	= 80.822%

  Hasil Perbandingan Dengan Data Sebenarnya
   Kata dasar benar tanpa imbuhan 				(benar) = 71
   Kata dasar hasil stemming  					(benar) = 43
   Bukan Kata Berimbuhan 					(salah) = 9
   Kata yang tidak berubah(Unchange) 				(salah) = 19
